In [1]:
from parameters.dataset import BPI2012Parameters
from dataset import BPI2012Dataset
from utils.print import print_block
from utils.bpi2012 import remove_trail_steps, print_model_prediction_result, remove_tags_for_seq
import tensorflow as tf
from model import LSTMPredNextModel, LSTMScenarioCfModel
import numpy as np
from dice import EventLogDiCE
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from utils.preprocessing import dataset_split
from cf_search import CfSearcher

In [2]:
dataset = BPI2012Dataset(BPI2012Parameters()) 


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [3]:
### Prepare the training dataset.
train_params = TrainingParameters(
    stop_epoch=20,
)
train_dataset, test_dataset, validation_dataset = dataset_split(
    list(range(len(dataset))),
    train_params.train_test_split_portion,
    seed=train_params.random_seed,
    shuffle=True
)
training_df = dataset.df.iloc[train_dataset]

In [4]:
#### Get the trace
trace_idx = 52
_, example_activities, _, example_resources, example_amount, _ = dataset.collate_fn([trace_idx])
print_block(dataset.activity_vocab.list_of_index_to_vocab_2d(example_activities),"Activity")
print_block(dataset.resource_vocab.list_of_index_to_vocab_2d(example_resources),"Resource")


====================Activity====================
| [['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_ACCEPTED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE', 'W_Valideren aanvraag_COMPLETE']] 

====================Resource====================
| [['<SOS>', '112', '112', '112', '11180', '11201', '11201', '11201', '11201', '11201', '11201', '11201', '11049', '11049', '10629', '10629', '10629', '10629', '10629']] 


In [5]:
## Remove tails
tail_length_to_remove = 13
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = dataset.activity_vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
example_vocab_resources = dataset.resource_vocab.list_of_index_to_vocab_2d(example_idx_resources)[0]
print_block(example_vocab_activities, "Activity (Tail removed)")
print_block(example_vocab_resources, "Resource (Tail removed)")

ground_truth_vocab =dataset.activity_vocab.index_to_vocab(example_activities[:, len(example_vocab_activities)][0])

print_block(ground_truth_vocab, "Ground Truth")


====================Activity (Tail removed)====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE'] 

====================Resource (Tail removed)====================
| ['<SOS>', '112', '112', '112', '11180', '11201'] 

====================Ground Truth====================
| A_ACCEPTED_COMPLETE 


In [6]:
pred_model = LSTMPredNextModel.load(
    "./SavedModels/%s" % (
    "0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Vocab loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Model loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  


In [7]:
_ = pred_model(**pred_model.get_example_input())
pred_model.summary()

Model: "LSTMPredNextModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (LSTM)                multiple                  49408     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33024     
_________________________________________________________________
sequential (Sequential)      (1, 64)             

In [21]:
example_idx_activities = dataset.activity_vocab.list_of_vocab_to_index_2d([[
    "<SOS>",
    "A_SUBMITTED_COMPLETE",
    "A_PARTLYSUBMITTED_COMPLETE",
    "A_PREACCEPTED_COMPLETE",
    "A_ACCEPTED_COMPLETE",
    "O_SELECTED_COMPLETE"
]])

example_idx_resources = dataset.resource_vocab.list_of_vocab_to_index_2d([[
    "<SOS>",
    "112",
    "112",
    "112",
    "10862",
    "10862",
]])

example_amount = [20000.0]

In [22]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(pred_model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (1.00) is "A_FINALIZED_COMPLETE" 



<PAD>                                      1.313926e-10
<EOS>                                      1.705847e-09
<SOS>                                      8.465242e-11
A_ACCEPTED_COMPLETE                        1.145822e-09
A_ACTIVATED_COMPLETE                       1.983463e-06
A_APPROVED_COMPLETE                        1.191397e-11
A_CANCELLED_COMPLETE                       7.438329e-08
A_DECLINED_COMPLETE                        6.775948e-07
A_FINALIZED_COMPLETE                       9.998353e-01
A_PARTLYSUBMITTED_COMPLETE                 2.523655e-11
A_PREACCEPTED_COMPLETE                     9.249423e-11
A_REGISTERED_COMPLETE                      1.066892e-06
A_SUBMITTED_COMPLETE                       3.652351e-08
O_ACCEPTED_COMPLETE                        9.841297e-12
O_CANCELLED_COMPLETE                 

In [9]:
searcher = CfSearcher(training_df, pred_model)

In [10]:
cf_df = searcher.search(example_vocab_activities, "A_ACCEPTED_COMPLETE", amount=example_amount[0])

In [12]:
cf_df

,activity,activity_vocab,caseid,amount,resource,resource_vocab,Predicted_vocab,Predicted_value
52,"[2, 12, 9, 10, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,15500.0,"[70, 53, 53, 53, 50, 55]","[<SOS>, 112, 112, 112, 11180, 11201]",W_Completeren aanvraag_COMPLETE,0.685416
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,15500.0,"[70, 53, 53, 53, 71, 55, 18, 18, 20]","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ...",W_Completeren aanvraag_COMPLETE,0.878228
6714,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,15500.0,"[70, 53, 53, 30, 30]","[<SOS>, 112, 112, 10939, 10939]",W_Completeren aanvraag_COMPLETE,0.528128
1031,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",176927,15500.0,"[70, 53, 53, 38, 38]","[<SOS>, 112, 112, 11009, 11009]",W_Completeren aanvraag_COMPLETE,0.671937
7405,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",196777,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.555065
583,"[2, 12, 9, 10, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",175528,15500.0,"[70, 53, 53, 53, 22]","[<SOS>, 112, 112, 112, 10912]",W_Completeren aanvraag_COMPLETE,0.652978
7554,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197234,15500.0,"[70, 53, 53, 25, 25]","[<SOS>, 112, 112, 10929, 10929]",W_Completeren aanvraag_COMPLETE,0.608703
9470,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",203206,15500.0,"[70, 53, 53, 21, 21]","[<SOS>, 112, 112, 10910, 10910]",W_Completeren aanvraag_COMPLETE,0.634169
7786,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197948,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.555065


In [14]:
cf_df.to_csv("result_1.csv")